### Import Module

In [3]:
import os
import glob
import numpy as np
import pandas as pd
import time
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow import feature_column
from tensorflow.keras import layers
import keras_tuner as kt
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
# from tensorflow.keras.layers.experimental import preprocessing
# from tensorflow.keras.utils import to_categorical
import os 
os.getcwd()
os.chdir('C:/Users/ASUS/Desktop/Github/Face_Recognition_with_mask')
os.getcwd()

'C:\\Users\\ASUS\\Desktop\\Github\\Face_Recognition_with_mask'

### Import Data

In [4]:
age_label_list = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-100']

file = open("C:/Users/ASUS/Desktop/Github/Face_Recognition_with_mask/Data/Train_dataset_50id_vector_Non_masked.txt",'r')
lines = file.readlines()
first = True
df = None
count = 1
vector_list = []
gender_list = []
race_list = []
age_list = []
id_list = []
for line in lines:
    data = line.split('|')
    image_id = line.split('/')[3]
    image_id = image_id[:image_id.find('\\')]
    id_list.append(image_id)
    list_of_vector = data[1].split(' ')
    list_of_vector = map(float, list_of_vector)
    list_of_vector = list(list_of_vector)
    vector_list.append(list_of_vector)
    list_age = data[2].split(' ')
    list_gender = data[3].split(' ')
    list_race = data[4].split(' ') 
    if np.argmax(list_gender) == 0:
        gender_list.append('Women')
#         print(str(count) + 'A')
    elif np.argmax(list_gender) == 1:
        gender_list.append('Men')   
    if np.argmax(list_race) == 0:
        race_list.append('Asian')
    elif np.argmax(list_race) == 1:
        race_list.append('Indian')
    elif np.argmax(list_race) == 2:
        race_list.append('Black')
    elif np.argmax(list_race) == 3:
        race_list.append('White')
    elif np.argmax(list_race) == 4:
        race_list.append('Middle_Eastern')
    elif np.argmax(list_race) == 5:
        race_list.append('Latino_Hispanic') 
#     print(list_age)
    list_age = map(float, list_age)
    list_age = list(list_age)
#     print(list_age)
    age_predictions = np.array(list_age)
#     print(age_predictions.dtype)
    output_indexes = np.array([i for i in range(0, 101)])
#     print(output_indexes.dtype)
    apparent_age = np.sum(age_predictions * output_indexes)
    apparent_age = round(apparent_age)
    if apparent_age >= 0 and apparent_age <= 9:
        age_list.append('0-9')
#         print('A')
    elif apparent_age >= 10 and apparent_age <= 19:
        age_list.append('10-19')
#         print('B')
    elif apparent_age >= 20 and apparent_age <= 29:
        age_list.append('20-29')
#         print('C')
    elif apparent_age >= 30 and apparent_age <= 39:
        age_list.append('30-39')
#         print('D')
    elif apparent_age >= 40 and apparent_age <= 49:
        age_list.append('40-49')
#         print('E')
    elif apparent_age >= 50 and apparent_age <= 59:
        age_list.append('50-59')
#         print('F')
    elif apparent_age >= 60 and apparent_age <= 69:
        age_list.append('60-69')
#         print('G')
    elif apparent_age >= 70 and apparent_age <= 79:
        age_list.append('70-79')
#         print('H')
    elif apparent_age >= 80 and apparent_age <= 89:                   
        age_list.append('80-89')
#         print('I')
    elif apparent_age >= 90 and apparent_age <= 100:
        age_list.append('90-100')
#         print('J')

#     print(apparent_age)
#     print(len(list_age))
#     print(count)
#     print('-=-=-=-=-=')
#     count += 1
#     vector = np.array(list_of_vector)
#     vector.reshape(-1, 2622, 1)
#     print(len(list_of_vector))
    
    
#     print(np.argmax(list_gender,axis=0))

# print(main_dict_list[0])
# dataframe_masked_train = pd.DataFrame.from_dict(train_dict_list_masked)
# print(len(gender_list))
# print(len(race_list))
# print(len(age_list))
dataframe_train = pd.DataFrame(vector_list)
one_hot_gender = pd.get_dummies(gender_list)
one_hot_race = pd.get_dummies(race_list)
one_hot_age = pd.get_dummies(age_list)

for i in range(10):
    if age_label_list[i] not in one_hot_age.columns:
        one_hot_age.insert(loc=i,column=age_label_list[i],value = 0)

le = preprocessing.LabelEncoder()
le.fit(id_list)
encoded_id_train = le.transform(id_list)

dataframe_train = dataframe_train.join(one_hot_age)
dataframe_train = dataframe_train.join(one_hot_gender)
dataframe_train = dataframe_train.join(one_hot_race)
dataframe_train['id'] = encoded_id_train
dataframe_train

,0,1,2,3,4,5,6,7,8,9,...,90-100,Men,Women,Asian,Black,Indian,Latino_Hispanic,Middle_Eastern,White,id
0,-0.001511,-0.007979,-0.021249,0.010664,-0.007335,-0.003255,0.012502,0.014529,-0.005273,0.009105,...,0,0,1,0,0,0,0,1,0,0
1,-0.002029,-0.007151,-0.017678,0.007096,-0.011775,-0.004272,0.004986,0.021701,-0.015278,0.006131,...,0,0,1,0,0,0,1,0,0,0
2,-0.002845,-0.010357,-0.010958,0.013247,0.004672,-0.001559,-0.012745,0.014603,-0.004680,0.005172,...,0,1,0,0,0,0,0,0,1,0
3,0.002517,-0.008903,-0.010357,0.005166,-0.001825,-0.004463,-0.000680,0.021333,0.002563,0.005380,...,0,0,1,0,0,0,0,0,1,0
4,-0.000587,0.001859,-0.009970,0.007103,0.002579,-0.000166,-0.000248,0.008783,-0.011554,0.006251,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8291,0.001518,-0.004695,-0.003033,0.022928,0.020972,0.010925,-0.001013,0.040531,0.010064,0.008347,...,0,0,1,0,0,0,0,0,1,49
8292,0.031696,-0.001501,-0.016892,0.020605,0.025902,0.013530,-0.003793,0.023630,0.002694,-0.008832,...,0,0,1,0,0,0,0,0,1,49
8293,0.018732,-0.009901,-0.009036,0.025380,0.026952,0.013855,-0.005313,0.026876,-0.002605,0.005254,...,0,0,1,0,0,0,0,0,1,49
8294,0.004887,0.008089,0.010522,0.017752,0.018814,0.007535,0.009224,0.027086,0.010898,0.016034,...,0,0,1,0,0,0,0,0,1,49


In [5]:
# C:/Users/ASUS/Desktop/Github/Face_Recognition_with_mask/Data/Test_dataset_50id_vector_masked.txt"

age_label_list = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-100']

file = open("C:/Users/ASUS/Desktop/Github/Face_Recognition_with_mask/Data/Test_dataset_50id_vector_Masked_Ver_2.txt",'r')
lines = file.readlines()
first = True
df = None
count = 1
vector_list = []
gender_list = []
race_list = []
age_list = []
id_list = []
for line in lines:
    data = line.split('|')
    image_id = line.split('/')[3]
    image_id = image_id[:image_id.find('\\')]
    id_list.append(image_id)
    list_of_vector = data[1].split(' ')
    list_of_vector = map(float, list_of_vector)
    list_of_vector = list(list_of_vector)
    vector_list.append(list_of_vector)
    list_age = data[2].split(' ')
    list_gender = data[3].split(' ')
    list_race = data[4].split(' ') 
    if np.argmax(list_gender) == 0:
        gender_list.append('Women')
    elif np.argmax(list_gender) == 1:
        gender_list.append('Men')   
    if np.argmax(list_race) == 0:
        race_list.append('Asian')
    elif np.argmax(list_race) == 1:
        race_list.append('Indian')
    elif np.argmax(list_race) == 2:
        race_list.append('Black')
    elif np.argmax(list_race) == 3:
        race_list.append('White')
    elif np.argmax(list_race) == 4:
        race_list.append('Middle_Eastern')
    elif np.argmax(list_race) == 5:
        race_list.append('Latino_Hispanic') 
    list_age = map(float, list_age)
    list_age = list(list_age)
    age_predictions = np.array(list_age)
    output_indexes = np.array([i for i in range(0, 101)])
    apparent_age = np.sum(age_predictions * output_indexes)
    apparent_age = round(apparent_age)
    if apparent_age >= 0 and apparent_age <= 9:
        age_list.append('0-9')
    elif apparent_age >= 10 and apparent_age <= 19:
        age_list.append('10-19')
    elif apparent_age >= 20 and apparent_age <= 29:
        age_list.append('20-29')
    elif apparent_age >= 30 and apparent_age <= 39:
        age_list.append('30-39')
    elif apparent_age >= 40 and apparent_age <= 49:
        age_list.append('40-49')
    elif apparent_age >= 50 and apparent_age <= 59:
        age_list.append('50-59')
    elif apparent_age >= 60 and apparent_age <= 69:
        age_list.append('60-69')
    elif apparent_age >= 70 and apparent_age <= 79:
        age_list.append('70-79')
    elif apparent_age >= 80 and apparent_age <= 89:                   
        age_list.append('80-89')
    elif apparent_age >= 90 and apparent_age <= 100:
        age_list.append('90-100')

dataframe_test = pd.DataFrame(vector_list)
one_hot_gender = pd.get_dummies(gender_list)
one_hot_race = pd.get_dummies(race_list)
one_hot_age = pd.get_dummies(age_list)

for i in range(10):
    if age_label_list[i] not in one_hot_age.columns:
        one_hot_age.insert(loc=i,column=age_label_list[i],value = 0)

# print(id_list)
le = preprocessing.LabelEncoder()
le.fit(id_list)
encoded_id_test = le.transform(id_list)        

dataframe_test = dataframe_test.join(one_hot_age)
dataframe_test = dataframe_test.join(one_hot_gender)
dataframe_test = dataframe_test.join(one_hot_race)
dataframe_test['id'] = encoded_id_test
dataframe_test

,0,1,2,3,4,5,6,7,8,9,...,90-100,Men,Women,Asian,Black,Indian,Latino_Hispanic,Middle_Eastern,White,id
0,-0.005811,0.006799,0.006527,0.006695,0.014906,0.011761,0.002441,0.014137,0.001551,0.000994,...,0,1,0,0,0,0,0,0,1,0
1,-0.000615,0.004171,0.008273,0.014302,0.010149,0.014710,0.005147,0.013600,0.005837,0.008584,...,0,0,1,0,0,0,0,0,1,0
2,-0.004168,-0.001410,0.005084,0.008728,0.003481,0.017883,-0.002308,0.017414,0.006853,0.008259,...,0,0,1,0,0,0,0,0,1,0
3,0.000304,0.011040,0.014343,0.013360,0.009079,0.017290,0.014120,0.020558,-0.001548,0.008252,...,0,0,1,0,0,0,0,0,1,0
4,-0.003455,-0.003399,0.012119,0.010312,0.004979,0.024713,0.004397,0.020256,0.018522,0.009038,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2082,0.014161,0.008186,0.017251,0.013830,0.030316,-0.001384,0.008802,0.016736,-0.002087,0.004245,...,0,0,1,0,0,0,0,0,1,49
2083,0.010095,0.012177,0.013166,0.010713,0.036137,0.000886,0.002403,0.028041,0.012473,0.016539,...,0,1,0,0,0,1,0,0,0,49
2084,0.014688,0.009354,0.012795,0.011314,0.033331,0.011004,0.000947,0.019660,0.006582,0.028804,...,0,0,1,0,0,0,0,0,1,49
2085,0.018985,0.005582,0.009959,0.016992,0.031971,0.009151,-0.000118,0.028676,0.006455,0.012933,...,0,1,0,0,0,0,0,0,1,49


### Preprocessing Data

In [6]:
age_label_list = ['0-9','10-19','20-29','30-39','40-49','50-59','60-69','70-79','80-89','90-100']
gender_label_list = ['Men','Women']
race_label_list = ['Asian','Black','Indian','Latino_Hispanic','Middle_Eastern','White']

all_features_label_list = age_label_list + gender_label_list + race_label_list
age_gender_label = race_label_list
age_race_label = gender_label_list
gender_race_label = age_label_list
age_label = gender_label_list + race_label_list
gender_label = age_label_list + race_label_list
race_label = age_label_list + gender_label_list

all_feature_activate = False
if all_feature_activate is not True:
    dataframe_train = dataframe_train.drop(all_features_label_list, axis=1)
    dataframe_test = dataframe_test.drop(all_features_label_list, axis=1)

train, val = train_test_split(dataframe_train, test_size=0.2, random_state=17)
print(len(train), 'train examples')
print(len(val), 'validation examples')
# print(len(dataframe_masked_test), 'test examples')

6636 train examples
1660 validation examples


In [7]:
train.head()

,0,1,2,3,4,5,6,7,8,9,...,2613,2614,2615,2616,2617,2618,2619,2620,2621,id
6626,0.009531,-0.000147,0.027317,0.002584,-0.005734,0.002536,-0.002560,0.022687,0.001713,0.007170,...,-0.003761,-0.009793,-0.003024,-0.001855,0.000413,-0.015434,-0.010990,-0.003005,0.004059,39
5155,0.035676,-0.003842,0.001944,0.007035,0.007077,0.011967,0.000635,0.032977,0.015107,0.003923,...,-0.019023,-0.027523,-0.022306,0.002054,-0.001775,-0.009504,-0.013008,-0.010018,-0.001794,31
7053,0.014986,-0.004960,-0.012233,0.021403,0.005923,0.009551,0.001746,0.013029,0.001418,0.009171,...,0.007736,-0.010128,-0.012123,-0.001020,-0.000024,-0.009618,-0.003140,-0.004004,-0.001260,42
6591,0.014594,-0.002820,0.019973,-0.004267,0.001499,0.000641,0.009370,0.020481,-0.002007,-0.008076,...,0.008998,-0.010725,0.001120,0.019741,-0.009193,-0.016483,-0.017217,-0.005673,0.008655,39
1158,0.005785,0.005078,0.000820,-0.002866,0.001802,0.005438,0.006097,0.010223,0.009175,-0.001732,...,-0.001233,0.004778,0.006424,0.004949,-0.004005,-0.004954,-0.008047,0.008832,0.015968,7


In [8]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('id')
#     labels = to_categorical(labels)
    ds = tf.data.Dataset.from_tensor_slices((dataframe, labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [9]:
train_ds = df_to_dataset(train)
test_ds = df_to_dataset(dataframe_test, shuffle=False)
val_ds = df_to_dataset(val, shuffle=False)

In [10]:
print(len(train_ds))
for feature_batch, label_batch in val_ds.take(1):
#     print(feature_batch)
#     print('Every feature:', list(feature_batch.keys()))
#     print('A batch of ages:', feature_batch['age'])
    print('A batch of targets:', label_batch )

208
A batch of targets: tf.Tensor(
[ 4 19 25 40 49 20  1 38 41 19 17 37 42 14 44  6  8 24 28 42  9  2 13 41
 43 34 42  2 35 38 37 49], shape=(32,), dtype=int32)


### Train

In [15]:
# 2 ^ N
model = Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=(2622,))) #2622 for non-features 2640 for all features 2634 for age and gender 2638 for age and race 2630 for gender and race
# model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dense(2048, activation='relu'))
model.add(tf.keras.layers.Dense(1024, activation='relu'))
# model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.Dense(128, activation='relu'))
# model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(50, activation='softmax')) #layer_number = id
#Add layer to check the accuracy

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 2048)              5371904   
_________________________________________________________________
dense_6 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dense_7 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_8 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_9 (Dense)              (None, 50)                6450      
Total params: 7,771,826
Trainable params: 7,771,826
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
    metrics = ['accuracy']
)

In [18]:
TIC = time.time()
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100
)
TOC = time.time()
print("It takes",TOC-TIC," seconds")

Epoch 1/100


C:\Users\ASUS\anaconda3\envs\env_dlib\lib\site-packages\tensorflow\python\keras\backend.py:4929: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  warnings.warn(


208/208 [==============================] - 2s 4ms/step - loss: 1.5685 - accuracy: 0.5868 - val_loss: 0.5622 - val_accuracy: 0.8494
Epoch 2/100
208/208 [==============================] - 1s 4ms/step - loss: 0.4128 - accuracy: 0.8858 - val_loss: 0.3471 - val_accuracy: 0.9024
Epoch 3/100
208/208 [==============================] - 1s 4ms/step - loss: 0.2700 - accuracy: 0.9234 - val_loss: 0.3356 - val_accuracy: 0.9096
Epoch 4/100
208/208 [==============================] - 1s 4ms/step - loss: 0.1985 - accuracy: 0.9448 - val_loss: 0.3352 - val_accuracy: 0.9084
Epoch 5/100
208/208 [==============================] - 1s 4ms/step - loss: 0.1606 - accuracy: 0.9543 - val_loss: 0.3447 - val_accuracy: 0.9060
Epoch 6/100
208/208 [==============================] - 1s 4ms/step - loss: 0.1258 - accuracy: 0.9620 - val_loss: 0.2874 - val_accuracy: 0.9223
Epoch 7/100
208/208 [==============================] - 1s 5ms/step - loss: 0.0989 - accuracy: 0.9708 - val_loss: 0.2859 - val_accuracy: 0.9301
Epoch 8/100

In [23]:
loss, accuracy = model.evaluate(test_ds)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

66/66 [==============================] - 0s 2ms/step - loss: 7.6069 - accuracy: 0.5625
Loss:  7.6068501472473145
Accuracy:  0.5625299215316772


### Keras Tuner

In [2]:
def build_model(hp):
    model = tf.keras.Sequential()
    model.add(layers.Flatten(input_shape=(2632,)))
    for i in range(hp.Int('layers', 1, 3)):
        model.add(layers.Dense(units=hp.Choice('units_' + str(i), values=[128,256,512,1024,2048]), activation='relu'))    
    model.add(tf.keras.layers.Dense(50, activation='softmax'))
    model.compile('adam', 'sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [75]:
tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials=100,
    project_name='kt_mixed_with_race',
    directory = 'D:/Tuning/Keras'   
)

INFO:tensorflow:Reloading Oracle from existing project D:/Tuning/Keras\kt_mixed_with_age\oracle.json


In [4]:
# stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [77]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 3, 'step': 1, 'sampling': None}
units_0 (Choice)
{'default': 128, 'conditions': [], 'values': [128, 256, 512, 1024, 2048], 'ordered': True}


In [78]:
tuner.search(train_ds, epochs=100, validation_data=val_ds)

Trial 50 Complete [00h 02m 54s]
val_accuracy: 0.918968141078949

Best val_accuracy So Far: 0.9262518882751465
Total elapsed time: 02h 41m 37s
INFO:tensorflow:Oracle triggered exit


In [42]:
# best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
# model = tuner.hypermodel.build(best_hps)
# history = model.fit(train_ds,validation_data=val_ds,epochs=100)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Best epoch: 73


In [ ]:
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]
hypermodel = tuner.hypermodel.build(best_hps)
history = hypermodel.fit(train_ds,validation_data=val_ds,epochs=100)

In [87]:
hypermodel.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 2632)              0         
_________________________________________________________________
dense (Dense)                (None, 2048)              5392384   
_________________________________________________________________
dense_1 (Dense)              (None, 2048)              4196352   
_________________________________________________________________
dense_2 (Dense)              (None, 50)                102450    
Total params: 9,691,186
Trainable params: 9,691,186
Non-trainable params: 0
_________________________________________________________________


In [92]:
loss, accuracy = hypermodel.evaluate(test_ds)
print("Loss: ", loss)
print("Accuracy: ", accuracy)
val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

66/66 [==============================] - 0s 2ms/step - loss: 1.8376 - accuracy: 0.8548
Loss:  1.837570309638977
Accuracy:  0.8548155426979065
Best epoch: 72


### Evaluate Model

In [238]:
loss, accuracy = model.evaluate(test_ds)
print("Loss: ", loss)
print("Accuracy: ", accuracy)

26/26 [==============================] - 0s 2ms/step - loss: 1.2691 - accuracy: 0.8958
Loss:  1.2690939903259277
Accuracy:  0.8958333134651184


In [21]:
# model.save('Non_masked_classification_For_Masked')
# reloaded_model = tf.keras.models.load_model('masked_classification')

INFO:tensorflow:Assets written to: Non_masked_classification_For_Masked\assets


In [72]:
# reloaded_model = tf.keras.models.load_model('Hyper_mixed_classification_with_age')
# loss, accuracy = reloaded_model.evaluate(test_ds)
# print("Loss: ", loss)
# print("Accuracy: ", accuracy)

66/66 [==============================] - 0s 2ms/step - loss: 1.1233 - accuracy: 0.8721
Loss:  1.123286247253418
Accuracy:  0.8720651865005493
